#### NAME: 
Joe (Joseph) Squire
#### ANSWERS
1. 3105 rows
2. 33 unique values

# Load Libraries

In [ ]:
import boto3
import pandas as pd

# Access & Download files locally

In [ ]:
# Create an S3 client
session = boto3.Session(profile_name='squirejr')
s3 = session.client('s3')

## Step 1 Data Transformation
Objective: transform dataset from patient_id x month_year granulatriy to patient_id x enrollment_start_date x enrollment_end_date

In [ ]:
# Download patient_id_month_year.csv
bucket_name = 'waymark-assignment'
s3_file_key = 'patient_id_month_year.csv'
local_file_path = 'patient_id_month_year.csv'

s3.download_file(bucket_name, s3_file_key, local_file_path)

### Load & Explore the data

In [ ]:
# Reading in CSV
# Defined dtypes since pd was throwing an warning of mixed dtypes in cols
# Defined what columns to use since pd was bringing in multiple empty columns
df = pd.read_csv('patient_id_month_year.csv', dtype={0: str, 1: str}, usecols=[0, 1])

In [ ]:
#See initial layout of dataframe
print(df.head())
print(df.head())
print(df.info())

In [ ]:
#Convert month_year to date type
df['month_year'] = pd.to_datetime(df['month_year'], format='%m/%d/%y')

# Clean df
# Remove rows where all cells in that row are NaN
df = df.dropna(how='all')

In [ ]:
#Reexamine imported df
print(df.head())
print(df.tail())
print(df.info())

In [ ]:
# Checks to see if every patient has a single enrollment start and end date
unique_counts = df['patient_id'].value_counts()

print(unique_counts)

# Output showing patients do not have continuous enrollment throughout the year
# Will need to account for patient having multiple enrollment periods in transformation

### Transform and group data

In [ ]:
# Add enrollment start date column
df['enrollment_start_date'] = df['month_year']

# Add enrollment end date colum as last day of each month listed
df['enrollment_end_date'] = df['month_year'] + pd.offsets.MonthEnd(1)

In [ ]:
# Sort the dataframe for operations below (assumptions are largely be based on this sort)
df = df.sort_values(by=['patient_id', 'enrollment_start_date'])

# Prepare a list to collect dict from for loop
rows_list = []

# Loop through each row belonging to the same patient based on patient_id
# Evaluates whether start date of a current row is +1 day from the last evaluated end date (i.e. they are consecutive)
# If this is true, the enrollment period is extended 
# If this is not true, a row is added to the dict that includes patient_id, start and end dates
for patient_id, group in df.groupby('patient_id'):
    group = group.reset_index(drop=True)
    start_date = group.at[0, 'enrollment_start_date']
    end_date = group.at[0, 'enrollment_end_date']

    for i in range(1, len(group)):
        if group.at[i, 'enrollment_start_date'] == end_date + pd.Timedelta(days=1):
            end_date = group.at[i, 'enrollment_end_date']
        else:
            rows_list.append({'patient_id': patient_id, 'enrollment_start_date': start_date, 'enrollment_end_date': end_date})
            start_date = group.at[i, 'enrollment_start_date']
            end_date = group.at[i, 'enrollment_end_date']

    # Add the last range to the list
    rows_list.append({'patient_id': patient_id, 'enrollment_start_date': start_date, 'enrollment_end_date': end_date})

# Create a new dataframe from the list
result_df = pd.DataFrame(rows_list)

# Ensure the dates are in the correct format
result_df['enrollment_start_date'] = pd.to_datetime(result_df['enrollment_start_date'], '%Y-%m-%d')
result_df['enrollment_end_date'] = pd.to_datetime(result_df['enrollment_end_date'], '%Y-%m-%d')

# Show the transformed DataFrame
print(result_df)

### Write to CSV

In [ ]:
result_df.to_csv('patient_enrollment_span.csv')

### Answer 1
Row Count: 3105

## Step 2 Data Aggregation

In [ ]:
# Download outpatient_visits_file.csv
bucket_name = 'waymark-assignment'
s3_file_key = 'outpatient_visits_file.csv'
local_file_path = 'outpatient_visits_file.csv'

s3.download_file(bucket_name, s3_file_key, local_file_path)

### Load and Explore Data

In [ ]:
# Reading in CSV
# Defined dtypes since pd was throwing an warning of mixed dtypes in cols
# Defined what columns to use since pd was bringing in multiple empty columns
visits_df = pd.read_csv('outpatient_visits_file.csv', dtype={0: str, 1: str}, usecols=[0, 1, 2])

In [ ]:
# Quick exploratory view
print(visits_df.head())
print(visits_df.tail())
print(visits_df.info())

In [ ]:
# Convert 'date' column to datetime format
visits_df['date'] = pd.to_datetime(visits_df['date'], format = '%m/%d/%y')

# Clean dataframe
# Remove rows where all cells in that row are NaN
visits_df = visits_df.dropna(how='all')

In [ ]:
# See initial layout of dataframe
print(visits_df.head())
print(visits_df.tail())
print(visits_df.info())

### Group data, summarize, & merge data

In [ ]:
# Initialize the columns for counts
result_df['ct_outpatient_visits'] = 0
result_df['ct_days_with_outpatient_visit'] = 0

# Iterate over each row in result_df to calculate the counts
# Filter visits_df for visits that fall within the enrollment period for the current patient
for idx, row in result_df.iterrows():
    valid_visits = visits_df[(visits_df['patient_id'] == row['patient_id']) &
                             (visits_df['date'] >= row['enrollment_start_date']) &
                             (visits_df['date'] <= row['enrollment_end_date'])]
    
    # Sum the outpatient visits count
    result_df.at[idx, 'ct_outpatient_visits'] = valid_visits['outpatient_visit_count'].sum()
    
    # Count distinct days with outpatient visits
    result_df.at[idx, 'ct_days_with_outpatient_visit'] = valid_visits['date'].nunique()

In [ ]:
# Quality check
# Looking for the same number of rows as found in step 1 plus ensuring accurate dtypes
result_df.info()

### Write to CSV

In [ ]:
# Write to CSV
result_df.to_csv('result.csv')

In [ ]:
# Counts number of unique values in ct_days_with_outpatient_visit
unique_count = result_df['ct_days_with_outpatient_visit'].nunique()

print(unique_count)

### Answer B
Unique values in ct_days_with_outpatient_visit: 33